In [1]:
## 
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import json

##
from DLinear import LTSF_DLinear
from DLinear import moving_avg
from jeju.utils.scalilng import standardization
from jeju.utils.scalilng import time_slide_df
from jeju.utils.scalilng import Data
from jeju.utils.scalilng import data_split
from jeju.utils.scalilng import date_range_to_numeric
from jeju.utils.scalilng import min_max_inverse_scaling
from jeju.utils.scalilng import min_max_scaling
from jeju.utils.scalilng import z_score
from jeju.utils.scalilng import z_score_inverse


/Users/hyeongyukang/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

# 상품, 회사, 지역 별로 분할하여 df_list에 저장
# unique code = {'CB_E_J', 'TG_A_S', 'TG_D_S', 'CB_A_S', 'RD_C_S', 'BC_E_J', 'CR_D_S', 'BC_C_J', 'RD_D_S', 'BC_A_J', 'BC_C_S', 'TG_C_J', 'TG_D_J', 
#                'TG_C_S', 'BC_D_J', 'TG_E_S', 'RD_D_J', 'CR_B_J', 'RD_A_J', 'TG_B_J', 'BC_A_S', 'CR_A_J', 'TG_B_S', 'CR_E_S', 'TG_E_J', 'CR_D_J', 
#                'TG_A_J', 'RD_E_J', 'BC_B_S', 'RD_F_J', 'CR_E_J', 'CB_A_J', 'BC_B_J', 'CR_C_J', 'CB_F_J', 'RD_E_S', 'RD_A_S', 'CB_D_J', 'BC_E_S'}

data_list = data_split(data)

unique_code = ['BC_C_J', 'TG_B_J', 'CR_B_J', 'RD_E_S', 'BC_A_J', 'CB_F_J', 'RD_D_J', 'TG_A_S', 'BC_E_S', 'CR_D_J', 'BC_A_S', 'BC_B_S', 'TG_E_J', 
               'CR_E_S', 'RD_F_J', 'BC_E_J', 'TG_A_J', 'CR_C_J', 'CR_D_S', 'TG_C_J', 'CB_A_S', 'TG_D_J', 'CR_E_J', 'RD_C_S', 'BC_C_S', 'CB_E_J', 
               'RD_E_J', 'BC_D_J', 'CR_A_J', 'TG_E_S', 'TG_C_S', 'TG_D_S', 'RD_A_S', 'RD_A_J', 'RD_D_S', 'TG_B_S', 'CB_D_J', 'CB_A_J', 'BC_B_J']


In [32]:
dataset_code = unique_code[16]
print(dataset_code)

train_df = data_list[f'data_{dataset_code}'].reset_index(drop=True)

# # scaling
# scaled_x, min_val, max_val = min_max_scaling(train_df['price(원/kg)'])
# train_df['price(원/kg)'] = scaled_x
    
# 날짜 변환
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])


TG_A_J


In [33]:
## paramaters
window_size = 7
forcast_size= 1
batch_size = 32
targets = ['supply(kg)','price(원/kg)']
date = 'timestamp'

In [34]:
def time_slide_df(df, window_size, forcast_size, date, target):
    df_ = df.copy()
    data_list = []
    dap_list = []
    date_list = []
    for idx in range(0, df_.shape[0]-window_size-forcast_size+1):
        x = df_.loc[idx:idx+window_size-1, target].values.reshape(window_size, len(target))
        y = df_.loc[idx+window_size:idx+window_size+forcast_size-1, target].values
        date_ = df_.loc[idx+window_size:idx+window_size+forcast_size-1, date].values
        data_list.append(x)
        dap_list.append(y)
        date_list.append(date_)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32'), np.array(date_list)


In [35]:
train_x, train_y, train_date = time_slide_df(train_df, window_size, forcast_size, date, targets)

In [36]:
print(train_x)

[[[    0.      0. ]
  [    0.      0. ]
  [60601.   1728. ]
  ...
  [32352.   1250. ]
  [    0.      0. ]
  [44995.   1474. ]]

 [[    0.      0. ]
  [60601.   1728. ]
  [25000.   1408. ]
  ...
  [    0.      0. ]
  [44995.   1474. ]
  [26975.   1326. ]]

 [[60601.   1728. ]
  [25000.   1408. ]
  [32352.   1250. ]
  ...
  [44995.   1474. ]
  [26975.   1326. ]
  [29265.   1428. ]]

 ...

 [[ 8022.   2813. ]
  [10094.8  2770. ]
  [10462.8  2633. ]
  ...
  [    0.      0. ]
  [ 7009.6  2657. ]
  [ 6775.   3922. ]]

 [[10094.8  2770. ]
  [10462.8  2633. ]
  [ 7115.   3155. ]
  ...
  [ 7009.6  2657. ]
  [ 6775.   3922. ]
  [ 7696.7  3397. ]]

 [[10462.8  2633. ]
  [ 7115.   3155. ]
  [    0.      0. ]
  ...
  [ 6775.   3922. ]
  [ 7696.7  3397. ]
  [ 5815.   3195. ]]]


In [40]:
print(train_y[-7:,:,:])

[[[ 7115.   3155. ]]

 [[    0.      0. ]]

 [[ 7009.6  2657. ]]

 [[ 6775.   3922. ]]

 [[ 7696.7  3397. ]]

 [[ 5815.   3195. ]]

 [[10343.6  3640. ]]]


In [41]:
import numpy as np

# 주어진 3차원 배열
original_array = np.array([[[7115., 3155.]],
                           [[0., 0.]],
                           [[7009.6, 2657.]],
                           [[6775., 3922.]],
                           [[7696.7, 3397.]],
                           [[5815., 3195.]],
                           [[10343.6, 3640.]]])

# 추가할 값
new_value = np.array([[[100., 200.]]])

# 새로운 값을 추가하여 새로운 배열 생성
result_array = np.concatenate((original_array, new_value), axis=0)

print(result_array)

[[[ 7115.   3155. ]]

 [[    0.      0. ]]

 [[ 7009.6  2657. ]]

 [[ 6775.   3922. ]]

 [[ 7696.7  3397. ]]

 [[ 5815.   3195. ]]

 [[10343.6  3640. ]]

 [[  100.    200. ]]]


In [42]:
original_array = np.array([[[7115., 3155.]],
                           [[0., 0.]],
                           [[7009.6, 2657.]],
                           [[6775., 3922.]],
                           [[7696.7, 3397.]],
                           [[5815., 3195.]],
                           [[10343.6, 3640.]],
                           [[100., 200.]]])

# 맨 처음 행을 제거한 새로운 배열 생성
result_array = original_array[1:]

print(result_array)

[[[    0.      0. ]]

 [[ 7009.6  2657. ]]

 [[ 6775.   3922. ]]

 [[ 7696.7  3397. ]]

 [[ 5815.   3195. ]]

 [[10343.6  3640. ]]

 [[  100.    200. ]]]


In [10]:
train_ds = Data(train_x[:1400], train_y[:1400])
valid_ds = Data(train_x[1400:], train_y[1400:])

# 앞에서부터 window_size = 7, stride = 1에 해당하는 윈도우들이 input
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle=False,)
valid_dl = DataLoader(valid_ds, batch_size = train_x[1400:].shape[0], shuffle=False)

In [11]:
data, target = next(iter(train_dl))
print(data, data.shape)

tensor([[[    0.,     0.],
         [    0.,     0.],
         [60601.,  1728.],
         [25000.,  1408.],
         [32352.,  1250.],
         [    0.,     0.],
         [44995.,  1474.]],

        [[    0.,     0.],
         [60601.,  1728.],
         [25000.,  1408.],
         [32352.,  1250.],
         [    0.,     0.],
         [44995.,  1474.],
         [26975.,  1326.]],

        [[60601.,  1728.],
         [25000.,  1408.],
         [32352.,  1250.],
         [    0.,     0.],
         [44995.,  1474.],
         [26975.,  1326.],
         [29265.,  1428.]],

        [[25000.,  1408.],
         [32352.,  1250.],
         [    0.,     0.],
         [44995.,  1474.],
         [26975.,  1326.],
         [29265.,  1428.],
         [21226.,  1433.]],

        [[32352.,  1250.],
         [    0.,     0.],
         [44995.,  1474.],
         [26975.,  1326.],
         [29265.,  1428.],
         [21226.,  1433.],
         [25752.,  1546.]],

        [[    0.,     0.],
         [44995., 

In [12]:
print(target, target.shape)

tensor([[[26975.,  1326.]],

        [[29265.,  1428.]],

        [[21226.,  1433.]],

        [[25752.,  1546.]],

        [[35488.,  1374.]],

        [[    0.,     0.]],

        [[15320.,  1694.]],

        [[40486.,  1456.]],

        [[37623.,  1470.]],

        [[ 3956.,  2919.]],

        [[39275.,  1374.]],

        [[22295.,  1330.]],

        [[    0.,     0.]],

        [[26366.,  2144.]],

        [[27582.,  2067.]],

        [[23029.,  2149.]],

        [[27157.,  2148.]],

        [[32280.,  2513.]],

        [[22324.,  2629.]],

        [[    0.,     0.]],

        [[26983.,  2932.]],

        [[24969.,  2472.]],

        [[27852.,  2377.]],

        [[15997.,  2278.]],

        [[36010.,  2349.]],

        [[ 8808.,  2286.]],

        [[    0.,     0.]],

        [[ 8468.,  1831.]],

        [[    0.,     0.]],

        [[    0.,     0.]],

        [[    0.,     0.]],

        [[    0.,     0.]]]) torch.Size([32, 1, 2])


In [13]:
class moving_avg(torch.nn.Module):
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = torch.nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

In [14]:
##
x = data
kernel_size = 25

##
ma = moving_avg(25,1)
x1 = ma(x)
print(x1)

tensor([[[17316.7207,   588.1600],
         [19116.5195,   647.1200],
         [20916.3203,   706.0800],
         [22716.1191,   765.0400],
         [24515.9199,   824.0000],
         [26315.7207,   882.9600],
         [28115.5195,   941.9200]],

        [[14070.9199,   605.6800],
         [15149.9199,   658.7200],
         [16228.9199,   711.7600],
         [17307.9199,   764.8000],
         [18386.9199,   817.8400],
         [19465.9199,   870.8800],
         [20544.9199,   923.9200]],

        [[44879.6016,  1516.7200],
         [43626.1602,  1504.7200],
         [42372.7188,  1492.7200],
         [41119.2812,  1480.7200],
         [39865.8398,  1468.7200],
         [38612.3984,  1456.7200],
         [37358.9609,  1444.7200]],

        [[24286.7598,  1352.5200],
         [24135.8008,  1353.5200],
         [23984.8398,  1354.5200],
         [23833.8809,  1355.5200],
         [23682.9199,  1356.5200],
         [23531.9609,  1357.5200],
         [23381.0000,  1358.5200]],

        [[28

In [15]:
class series_decomp(torch.nn.Module):
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        residual = x - moving_mean
        return moving_mean, residual 

In [16]:
decomposer = series_decomp(25)
moving_mean, residual = decomposer(x)
print(moving_mean[0])
print(residual[0])

tensor([[17316.7207,   588.1600],
        [19116.5195,   647.1200],
        [20916.3203,   706.0800],
        [22716.1191,   765.0400],
        [24515.9199,   824.0000],
        [26315.7207,   882.9600],
        [28115.5195,   941.9200]])
tensor([[-17316.7207,   -588.1600],
        [-19116.5195,   -647.1200],
        [ 39684.6797,   1021.9200],
        [  2283.8809,    642.9600],
        [  7836.0801,    426.0000],
        [-26315.7207,   -882.9600],
        [ 16879.4805,    532.0800]])


In [17]:
class LTSF_DLinear(torch.nn.Module):
    def __init__(self, window_size, forcast_size, kernel_size, individual, feature_size):
        super(LTSF_DLinear, self).__init__()
        self.window_size = window_size
        self.forcast_size = forcast_size
        self.decompsition = series_decomp(kernel_size)
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear_Seasonal = torch.nn.ModuleList()
            self.Linear_Trend = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear_Trend.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Trend[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
                self.Linear_Seasonal.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Seasonal[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
                self.Linear_Seasonal.append(torch.nn.Linear(self.forcast_size, self.forcast_size))
        else:
            self.Linear_Trend = torch.nn.Linear(self.window_size, self.forcast_size)
            self.Linear_Trend.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
            self.Linear_Seasonal = torch.nn.Linear(self.window_size,  self.forcast_size)
            self.Linear_Seasonal.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))

        self.Linear_Supply = torch.nn.Linear(self.forcast_size, self.forcast_size)
        self.Linear_Trend.weight = torch.nn.Parameter((1/100)*torch.ones([self.window_size, self.window_size]))

            


    def forward(self, x):
        trend_init, seasonal_init = self.decompsition(x)
        trend_init, seasonal_init = trend_init.permute(0,2,1), seasonal_init.permute(0,2,1)
        if self.individual:
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.forcast_size], dtype=trend_init.dtype).to(trend_init.device)
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.forcast_size], dtype=seasonal_init.dtype).to(seasonal_init.device)
            for idx in range(self.channels):
                trend_output[:, idx, :] = self.Linear_Trend[idx](trend_init[:, idx, :])
                seasonal_output[:, idx, :] = self.Linear_Seasonal[idx](seasonal_init[:, idx, :])                
        else:
            trend_output = self.Linear_Trend(trend_init)
            seasonal_output = self.Linear_Seasonal(seasonal_init)


        # x = seasonal_output[:,0,:] + trend_output[:,0,:] + seasonal_output[:,1,:] + trend_output[:,1,:]
        # return x
        x = seasonal_output + trend_output
        return x.permute(0,2,1)
        


In [18]:
DL = LTSF_DLinear(window_size, forcast_size, kernel_size, individual = True, feature_size = 2)
output = DL(x)

In [19]:
print(output)

tensor([[[23277.9824,   836.5918]],

        [[27131.5547,  1026.0204]],

        [[31312.2695,  1230.0204]],

        [[25687.2676,  1187.8774]],

        [[25794.6953,  1207.5918]],

        [[26242.6953,  1225.3062]],

        [[26242.6973,  1225.3062]],

        [[22003.4121,  1256.7346]],

        [[23933.5547,  1275.3060]],

        [[25127.5547,  1281.3062]],

        [[22660.4121,  1493.5918]],

        [[24592.2695,  1469.0204]],

        [[22707.5527,  1462.7346]],

        [[22707.5547,  1462.7346]],

        [[24285.5547,  1527.0203]],

        [[22442.1270,  1614.3060]],

        [[20357.2676,  1711.3062]],

        [[23671.6953,  1601.1633]],

        [[22672.4102,  1763.8776]],

        [[22676.5527,  1949.4490]],

        [[22676.5547,  1949.4490]],

        [[22764.6973,  2062.0205]],

        [[22391.4102,  2119.8777]],

        [[23080.4121,  2152.4490]],

        [[21486.1250,  2171.0205]],

        [[22018.9824,  2147.5918]],

        [[20088.1250,  2098.5918]],

 

tensor([[23278.2852],
        [27131.8574],
        [31312.5684],
        [25687.5703],
        [25794.9980],
        [26242.9980],
        [26243.0000],
        [22003.7148],
        [23933.8574],
        [25127.8574],
        [22660.7148],
        [24592.5684],
        [22707.8555],
        [22707.8555],
        [24285.8574],
        [22442.4297],
        [20357.5703],
        [23672.0000],
        [22672.7148],
        [22676.8555],
        [22676.8555],
        [22765.0000],
        [22391.7129],
        [23080.7148],
        [21486.4297],
        [22019.2852],
        [20088.4258],
        [20088.4277],
        [17443.4277],
        [13876.4277],
        [ 9897.5693],
        [ 7612.2842]], grad_fn=<SliceBackward0>)


In [22]:
pred_supply['TG_A_J']

array([7448.79541215, 7003.79691058, 7870.91336736, 7915.0487285 ,
       8062.02786985, 8084.24945301, 8524.58969167, 8229.32215312,
       8260.42023075, 8403.90621683, 8453.58749032, 8514.47872502,
       8568.21999219, 8637.02528825, 8634.4544647 , 8671.12013111,
       8714.79053301, 8746.50530481, 8778.00656387, 8806.87197012,
       8833.10260737, 8851.56983438, 8873.47602561, 8894.5259982 ,
       8913.14496017, 8930.63783205, 8946.6935572 , 8961.26986662])